# Module 08: Backtesting Frameworks

**Difficulty**: ⭐⭐⭐  
**Estimated Time**: 100 minutes  
**Prerequisites**: 
- [Module 03: Moving Averages](03_moving_averages.ipynb)
- [Module 04: RSI and Oscillators](04_rsi_oscillators.ipynb)
- [Module 05: MACD](05_macd.ipynb)

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** the importance of backtesting before deploying live trading strategies
2. **Implement** a simple backtesting framework from scratch in Python
3. **Backtest** the SMA 10 + 2% filter strategy on Malaysian stocks (achieving 24-35% returns)
4. **Backtest** the RSI + MACD combined strategy (targeting 73% win rate)
5. **Calculate** key performance metrics: win rate, profit factor, Sharpe ratio, and maximum drawdown
6. **Incorporate** realistic Malaysian transaction costs (RM2.88 minimum, 0.42% commission, stamp duty)
7. **Apply** walk-forward testing to avoid overfitting and curve-fitting
8. **Optimize** strategy parameters while understanding the risks of over-optimization

## Setup

Import all necessary libraries and configure the environment.

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Data fetching
import yfinance as yf

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Technical indicators
import ta

# Warning suppression
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)

# Configure visualization
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 7)

print("✓ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 1. Introduction to Backtesting

### What is Backtesting?

**Backtesting** is the process of testing a trading strategy on historical data to evaluate its performance before deploying it with real money. It answers the critical question: *"If I had used this strategy in the past, would it have been profitable?"*

### Why Backtesting Matters

1. **Risk Management**: Identify potential losses before they happen
2. **Strategy Validation**: Separate good ideas from bad ones
3. **Performance Expectations**: Set realistic return and risk targets
4. **Confidence Building**: Trade with conviction based on historical evidence

### Common Backtesting Pitfalls (Avoid These!)

| Pitfall | Description | Impact |
|---------|-------------|--------|
| **Look-Ahead Bias** | Using future information in past decisions | Unrealistically high returns |
| **Survivorship Bias** | Only testing on stocks that still exist today | Ignores delisted/failed companies |
| **Overfitting** | Optimizing parameters too much on historical data | Strategy fails on new data |
| **Ignoring Costs** | Not including commissions, slippage, fees | Profitable backtest, losing live trading |
| **Cherry-Picking** | Only showing best results or time periods | False confidence |

### The Golden Rule

> **"Past performance does not guarantee future results."**  
> Backtesting shows what *could have* happened, not what *will* happen.

## 2. Simple Backtesting Framework

Let's build a basic backtesting framework from scratch. We'll start with a **buy-and-hold** strategy as our baseline.

In [ ]:
# Fetch Malaysian stock data for backtesting
# Using Maybank (1155.KL) as our primary example

def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetch historical stock data from Yahoo Finance.
    
    Parameters:
    - ticker: Stock symbol (e.g., '1155.KL' for Maybank)
    - start_date: Start date for historical data
    - end_date: End date for historical data
    
    Returns:
    - DataFrame with OHLCV data
    """
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    return data

# Define our backtesting period (3 years of data)
start_date = '2021-01-01'
end_date = '2024-01-01'

# Fetch Maybank data
maybank_data = fetch_stock_data('1155.KL', start_date, end_date)

print(f"Data fetched: {len(maybank_data)} trading days")
print(f"Date range: {maybank_data.index[0]} to {maybank_data.index[-1]}")
maybank_data.head()

In [ ]:
# Create a simple Buy-and-Hold backtest as baseline

def backtest_buy_and_hold(data, initial_capital=10000):
    """
    Backtest a simple buy-and-hold strategy.
    
    Strategy: Buy on first day, hold until last day.
    
    Parameters:
    - data: DataFrame with 'Close' prices
    - initial_capital: Starting capital in RM
    
    Returns:
    - Dictionary with performance metrics
    """
    # Calculate number of shares we can buy
    entry_price = data['Close'].iloc[0]
    shares = initial_capital / entry_price
    
    # Calculate portfolio value over time
    portfolio_value = data['Close'] * shares
    
    # Calculate returns
    total_return = (portfolio_value.iloc[-1] - initial_capital) / initial_capital * 100
    final_value = portfolio_value.iloc[-1]
    
    return {
        'strategy': 'Buy and Hold',
        'initial_capital': initial_capital,
        'final_value': final_value,
        'total_return': total_return,
        'portfolio_value': portfolio_value
    }

# Run buy-and-hold backtest
bnh_results = backtest_buy_and_hold(maybank_data, initial_capital=10000)

print("=" * 60)
print("BUY-AND-HOLD BACKTEST RESULTS (Maybank 2021-2024)")
print("=" * 60)
print(f"Initial Capital: RM{bnh_results['initial_capital']:,.2f}")
print(f"Final Value: RM{bnh_results['final_value']:,.2f}")
print(f"Total Return: {bnh_results['total_return']:.2f}%")
print("=" * 60)

In [ ]:
# Visualize buy-and-hold performance

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Stock price over time
ax1.plot(maybank_data.index, maybank_data['Close'], label='Maybank Price', linewidth=2)
ax1.axhline(y=maybank_data['Close'].iloc[0], color='green', linestyle='--', 
            label=f'Entry: RM{maybank_data["Close"].iloc[0]:.2f}', alpha=0.7)
ax1.set_title('Maybank Stock Price (2021-2024)', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price (RM)', fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Portfolio value over time
ax2.plot(maybank_data.index, bnh_results['portfolio_value'], 
         label='Portfolio Value', color='green', linewidth=2)
ax2.axhline(y=10000, color='red', linestyle='--', 
            label='Initial Capital: RM10,000', alpha=0.7)
ax2.fill_between(maybank_data.index, 10000, bnh_results['portfolio_value'], 
                  where=(bnh_results['portfolio_value'] >= 10000), 
                  color='green', alpha=0.2, label='Profit')
ax2.fill_between(maybank_data.index, 10000, bnh_results['portfolio_value'], 
                  where=(bnh_results['portfolio_value'] < 10000), 
                  color='red', alpha=0.2, label='Loss')
ax2.set_title('Buy-and-Hold Portfolio Value', fontsize=16, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Portfolio Value (RM)', fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Strategy 1: SMA 10 + 2% Filter Backtest

Now let's implement a more sophisticated strategy based on **Malaysian research findings**:

### Strategy Rules:
1. **Buy Signal**: Price crosses above 10-day SMA AND price is at least 2% above SMA
2. **Sell Signal**: Price crosses below 10-day SMA
3. **Position Sizing**: Use all available capital for each trade

### Expected Performance (Malaysian Research):
- Historical returns: **24-35%** on Malaysian blue-chip stocks
- Works best in trending markets
- The 2% filter reduces false signals significantly

In [ ]:
def backtest_sma_filter_strategy(data, sma_period=10, filter_pct=2.0, initial_capital=10000):
    """
    Backtest SMA crossover strategy with percentage filter.
    
    Strategy:
    - BUY: Price > SMA(10) AND Price > SMA(10) * 1.02
    - SELL: Price < SMA(10)
    
    Parameters:
    - data: DataFrame with OHLCV data
    - sma_period: Period for Simple Moving Average
    - filter_pct: Percentage filter (2.0 means 2%)
    - initial_capital: Starting capital in RM
    
    Returns:
    - DataFrame with signals and portfolio value
    - Dictionary with performance metrics
    """
    df = data.copy()
    
    # Calculate SMA
    df['SMA'] = df['Close'].rolling(window=sma_period).mean()
    
    # Calculate filter threshold (SMA + 2%)
    df['Filter_Threshold'] = df['SMA'] * (1 + filter_pct / 100)
    
    # Generate signals
    df['Position'] = 0  # 0 = no position, 1 = long position
    
    # Buy when price crosses above filter threshold
    buy_condition = (df['Close'] > df['Filter_Threshold']) & \
                    (df['Close'].shift(1) <= df['Filter_Threshold'].shift(1))
    
    # Sell when price crosses below SMA
    sell_condition = (df['Close'] < df['SMA']) & \
                     (df['Close'].shift(1) >= df['SMA'].shift(1))
    
    # Track position status
    position = 0
    for i in range(len(df)):
        if buy_condition.iloc[i] and position == 0:
            position = 1  # Enter long position
        elif sell_condition.iloc[i] and position == 1:
            position = 0  # Exit position
        df.iloc[i, df.columns.get_loc('Position')] = position
    
    # Calculate strategy returns
    df['Strategy_Return'] = df['Close'].pct_change() * df['Position'].shift(1)
    df['Cumulative_Strategy_Return'] = (1 + df['Strategy_Return']).cumprod()
    df['Portfolio_Value'] = initial_capital * df['Cumulative_Strategy_Return']
    
    # Calculate trades
    df['Signal'] = df['Position'].diff()
    buy_signals = df[df['Signal'] == 1]
    sell_signals = df[df['Signal'] == -1]
    
    # Performance metrics
    total_return = (df['Portfolio_Value'].iloc[-1] - initial_capital) / initial_capital * 100
    num_trades = len(buy_signals)
    
    metrics = {
        'strategy': f'SMA({sma_period}) + {filter_pct}% Filter',
        'initial_capital': initial_capital,
        'final_value': df['Portfolio_Value'].iloc[-1],
        'total_return': total_return,
        'num_trades': num_trades,
        'buy_signals': buy_signals,
        'sell_signals': sell_signals
    }
    
    return df, metrics

# Run SMA strategy backtest
sma_df, sma_metrics = backtest_sma_filter_strategy(
    maybank_data, 
    sma_period=10, 
    filter_pct=2.0, 
    initial_capital=10000
)

print("=" * 60)
print("SMA 10 + 2% FILTER STRATEGY BACKTEST (Maybank 2021-2024)")
print("=" * 60)
print(f"Strategy: {sma_metrics['strategy']}")
print(f"Initial Capital: RM{sma_metrics['initial_capital']:,.2f}")
print(f"Final Value: RM{sma_metrics['final_value']:,.2f}")
print(f"Total Return: {sma_metrics['total_return']:.2f}%")
print(f"Number of Trades: {sma_metrics['num_trades']}")
print("=" * 60)
print(f"\nComparison with Buy-and-Hold:")
print(f"Buy-and-Hold Return: {bnh_results['total_return']:.2f}%")
print(f"Strategy Return: {sma_metrics['total_return']:.2f}%")
print(f"Difference: {sma_metrics['total_return'] - bnh_results['total_return']:.2f}%")

In [ ]:
# Visualize SMA strategy performance

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12))

# Plot 1: Price with SMA and signals
ax1.plot(sma_df.index, sma_df['Close'], label='Price', linewidth=2, color='black')
ax1.plot(sma_df.index, sma_df['SMA'], label='SMA(10)', linewidth=1.5, 
         color='blue', linestyle='--', alpha=0.7)
ax1.plot(sma_df.index, sma_df['Filter_Threshold'], label='Filter (+2%)', 
         linewidth=1.5, color='orange', linestyle=':', alpha=0.7)

# Mark buy and sell signals
if len(sma_metrics['buy_signals']) > 0:
    ax1.scatter(sma_metrics['buy_signals'].index, 
                sma_metrics['buy_signals']['Close'],
                color='green', marker='^', s=200, label='Buy Signal', zorder=5)
if len(sma_metrics['sell_signals']) > 0:
    ax1.scatter(sma_metrics['sell_signals'].index, 
                sma_metrics['sell_signals']['Close'],
                color='red', marker='v', s=200, label='Sell Signal', zorder=5)

ax1.set_title('SMA 10 + 2% Filter Strategy - Trading Signals', 
              fontsize=16, fontweight='bold')
ax1.set_ylabel('Price (RM)', fontsize=12)
ax1.legend(fontsize=10, loc='best')
ax1.grid(True, alpha=0.3)

# Plot 2: Portfolio value comparison
ax2.plot(sma_df.index, sma_df['Portfolio_Value'], 
         label='SMA Strategy', linewidth=2, color='blue')
ax2.plot(bnh_results['portfolio_value'].index, bnh_results['portfolio_value'], 
         label='Buy-and-Hold', linewidth=2, color='green', alpha=0.7)
ax2.axhline(y=10000, color='red', linestyle='--', 
            label='Initial Capital', alpha=0.7)

ax2.set_title('Portfolio Value: SMA Strategy vs Buy-and-Hold', 
              fontsize=16, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Portfolio Value (RM)', fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Strategy 2: RSI + MACD Combined Strategy

Let's implement a more complex strategy combining **RSI** and **MACD** indicators.

### Strategy Rules:
1. **Buy Signal**: 
   - RSI < 30 (oversold) AND
   - MACD line crosses above Signal line (bullish crossover)
2. **Sell Signal**:
   - RSI > 70 (overbought) OR
   - MACD line crosses below Signal line (bearish crossover)

### Expected Performance:
- Target **73% win rate** based on combined signals
- Reduces false signals by requiring confirmation from both indicators
- Works well in ranging and trending markets

In [ ]:
def backtest_rsi_macd_strategy(data, initial_capital=10000):
    """
    Backtest RSI + MACD combined strategy.
    
    Strategy:
    - BUY: RSI < 30 AND MACD crosses above Signal
    - SELL: RSI > 70 OR MACD crosses below Signal
    
    Parameters:
    - data: DataFrame with OHLCV data
    - initial_capital: Starting capital in RM
    
    Returns:
    - DataFrame with signals and portfolio value
    - Dictionary with performance metrics
    """
    df = data.copy()
    
    # Calculate RSI (14 periods)
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
    
    # Calculate MACD
    macd_indicator = ta.trend.MACD(df['Close'])
    df['MACD'] = macd_indicator.macd()
    df['MACD_Signal'] = macd_indicator.macd_signal()
    df['MACD_Diff'] = macd_indicator.macd_diff()
    
    # Generate signals
    df['Position'] = 0
    
    # MACD crossovers
    df['MACD_Cross_Up'] = (df['MACD'] > df['MACD_Signal']) & \
                          (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1))
    df['MACD_Cross_Down'] = (df['MACD'] < df['MACD_Signal']) & \
                            (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1))
    
    # Buy condition: RSI oversold + MACD bullish crossover
    buy_condition = (df['RSI'] < 30) & df['MACD_Cross_Up']
    
    # Sell condition: RSI overbought OR MACD bearish crossover
    sell_condition = (df['RSI'] > 70) | df['MACD_Cross_Down']
    
    # Track position status
    position = 0
    trades = []
    entry_price = 0
    
    for i in range(len(df)):
        if buy_condition.iloc[i] and position == 0:
            position = 1  # Enter long position
            entry_price = df['Close'].iloc[i]
            trades.append({'Date': df.index[i], 'Type': 'BUY', 'Price': entry_price})
        elif sell_condition.iloc[i] and position == 1:
            position = 0  # Exit position
            exit_price = df['Close'].iloc[i]
            trades.append({
                'Date': df.index[i], 
                'Type': 'SELL', 
                'Price': exit_price,
                'Return': (exit_price - entry_price) / entry_price * 100
            })
        df.iloc[i, df.columns.get_loc('Position')] = position
    
    # Calculate strategy returns
    df['Strategy_Return'] = df['Close'].pct_change() * df['Position'].shift(1)
    df['Cumulative_Strategy_Return'] = (1 + df['Strategy_Return']).cumprod()
    df['Portfolio_Value'] = initial_capital * df['Cumulative_Strategy_Return']
    
    # Calculate performance metrics
    df['Signal'] = df['Position'].diff()
    buy_signals = df[df['Signal'] == 1]
    sell_signals = df[df['Signal'] == -1]
    
    # Calculate win rate from completed trades
    trade_df = pd.DataFrame(trades)
    sell_trades = trade_df[trade_df['Type'] == 'SELL']
    if len(sell_trades) > 0:
        winning_trades = len(sell_trades[sell_trades['Return'] > 0])
        win_rate = (winning_trades / len(sell_trades)) * 100
    else:
        win_rate = 0
    
    total_return = (df['Portfolio_Value'].iloc[-1] - initial_capital) / initial_capital * 100
    
    metrics = {
        'strategy': 'RSI + MACD Combined',
        'initial_capital': initial_capital,
        'final_value': df['Portfolio_Value'].iloc[-1],
        'total_return': total_return,
        'num_trades': len(buy_signals),
        'win_rate': win_rate,
        'buy_signals': buy_signals,
        'sell_signals': sell_signals,
        'trades': trade_df
    }
    
    return df, metrics

# Run RSI + MACD strategy backtest
rsi_macd_df, rsi_macd_metrics = backtest_rsi_macd_strategy(maybank_data, initial_capital=10000)

print("=" * 60)
print("RSI + MACD COMBINED STRATEGY BACKTEST (Maybank 2021-2024)")
print("=" * 60)
print(f"Strategy: {rsi_macd_metrics['strategy']}")
print(f"Initial Capital: RM{rsi_macd_metrics['initial_capital']:,.2f}")
print(f"Final Value: RM{rsi_macd_metrics['final_value']:,.2f}")
print(f"Total Return: {rsi_macd_metrics['total_return']:.2f}%")
print(f"Number of Trades: {rsi_macd_metrics['num_trades']}")
print(f"Win Rate: {rsi_macd_metrics['win_rate']:.2f}%")
print("=" * 60)

# Display trade history
if len(rsi_macd_metrics['trades']) > 0:
    print("\nTrade History (Last 10 trades):")
    print(rsi_macd_metrics['trades'].tail(10).to_string(index=False))

In [ ]:
# Visualize RSI + MACD strategy

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(14, 16))

# Plot 1: Price with signals
ax1.plot(rsi_macd_df.index, rsi_macd_df['Close'], label='Price', 
         linewidth=2, color='black')
if len(rsi_macd_metrics['buy_signals']) > 0:
    ax1.scatter(rsi_macd_metrics['buy_signals'].index, 
                rsi_macd_metrics['buy_signals']['Close'],
                color='green', marker='^', s=200, label='Buy Signal', zorder=5)
if len(rsi_macd_metrics['sell_signals']) > 0:
    ax1.scatter(rsi_macd_metrics['sell_signals'].index, 
                rsi_macd_metrics['sell_signals']['Close'],
                color='red', marker='v', s=200, label='Sell Signal', zorder=5)
ax1.set_title('RSI + MACD Strategy - Trading Signals', fontsize=16, fontweight='bold')
ax1.set_ylabel('Price (RM)', fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: RSI
ax2.plot(rsi_macd_df.index, rsi_macd_df['RSI'], label='RSI', linewidth=1.5)
ax2.axhline(y=70, color='red', linestyle='--', label='Overbought (70)', alpha=0.7)
ax2.axhline(y=30, color='green', linestyle='--', label='Oversold (30)', alpha=0.7)
ax2.fill_between(rsi_macd_df.index, 30, 70, alpha=0.1)
ax2.set_title('Relative Strength Index (RSI)', fontsize=14)
ax2.set_ylabel('RSI', fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 100)

# Plot 3: MACD
ax3.plot(rsi_macd_df.index, rsi_macd_df['MACD'], label='MACD', linewidth=1.5)
ax3.plot(rsi_macd_df.index, rsi_macd_df['MACD_Signal'], 
         label='Signal', linewidth=1.5, alpha=0.7)
ax3.bar(rsi_macd_df.index, rsi_macd_df['MACD_Diff'], label='Histogram', alpha=0.3)
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax3.set_title('MACD Indicator', fontsize=14)
ax3.set_ylabel('MACD', fontsize=12)
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)

# Plot 4: Portfolio value
ax4.plot(rsi_macd_df.index, rsi_macd_df['Portfolio_Value'], 
         label='RSI+MACD Strategy', linewidth=2, color='purple')
ax4.plot(bnh_results['portfolio_value'].index, bnh_results['portfolio_value'], 
         label='Buy-and-Hold', linewidth=2, color='green', alpha=0.7)
ax4.axhline(y=10000, color='red', linestyle='--', label='Initial Capital', alpha=0.7)
ax4.set_title('Portfolio Value Comparison', fontsize=14)
ax4.set_xlabel('Date', fontsize=12)
ax4.set_ylabel('Portfolio Value (RM)', fontsize=12)
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Performance Metrics Calculation

A proper backtest must include comprehensive performance metrics beyond just total return.

### Key Metrics:

1. **Win Rate**: Percentage of profitable trades
2. **Profit Factor**: Gross profits / Gross losses (>1.5 is good)
3. **Sharpe Ratio**: Risk-adjusted return (>1.0 is good, >2.0 is excellent)
4. **Maximum Drawdown**: Largest peak-to-trough decline (lower is better)
5. **Average Trade**: Average profit/loss per trade
6. **Expectancy**: Expected value per trade

In [ ]:
def calculate_performance_metrics(df, metrics_dict, risk_free_rate=0.03):
    """
    Calculate comprehensive performance metrics for a backtest.
    
    Parameters:
    - df: DataFrame with backtest results
    - metrics_dict: Dictionary from backtest with basic metrics
    - risk_free_rate: Annual risk-free rate (3% for Malaysia)
    
    Returns:
    - Dictionary with comprehensive metrics
    """
    # Extract trade data
    if 'trades' in metrics_dict and len(metrics_dict['trades']) > 0:
        trade_df = metrics_dict['trades']
        sell_trades = trade_df[trade_df['Type'] == 'SELL'].copy()
        
        if len(sell_trades) > 0:
            # Win rate
            winning_trades = sell_trades[sell_trades['Return'] > 0]
            losing_trades = sell_trades[sell_trades['Return'] <= 0]
            win_rate = (len(winning_trades) / len(sell_trades)) * 100
            
            # Profit factor
            gross_profit = winning_trades['Return'].sum()
            gross_loss = abs(losing_trades['Return'].sum())
            profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
            
            # Average trade
            avg_win = winning_trades['Return'].mean() if len(winning_trades) > 0 else 0
            avg_loss = losing_trades['Return'].mean() if len(losing_trades) > 0 else 0
            avg_trade = sell_trades['Return'].mean()
            
            # Expectancy (expected value per trade)
            expectancy = (win_rate/100 * avg_win) + ((100-win_rate)/100 * avg_loss)
        else:
            win_rate = 0
            profit_factor = 0
            avg_win = 0
            avg_loss = 0
            avg_trade = 0
            expectancy = 0
    else:
        win_rate = 0
        profit_factor = 0
        avg_win = 0
        avg_loss = 0
        avg_trade = 0
        expectancy = 0
    
    # Calculate Sharpe Ratio
    # Assuming daily returns, annualize by multiplying by sqrt(252)
    strategy_returns = df['Strategy_Return'].dropna()
    if len(strategy_returns) > 0 and strategy_returns.std() > 0:
        excess_return = strategy_returns.mean() - (risk_free_rate / 252)  # Daily risk-free rate
        sharpe_ratio = (excess_return / strategy_returns.std()) * np.sqrt(252)
    else:
        sharpe_ratio = 0
    
    # Calculate Maximum Drawdown
    portfolio_value = df['Portfolio_Value']
    cumulative_max = portfolio_value.cummax()
    drawdown = (portfolio_value - cumulative_max) / cumulative_max
    max_drawdown = drawdown.min() * 100  # Convert to percentage
    
    # Calculate annualized return
    days_in_backtest = (df.index[-1] - df.index[0]).days
    years = days_in_backtest / 365.25
    total_return = metrics_dict['total_return']
    annualized_return = ((1 + total_return/100) ** (1/years) - 1) * 100
    
    return {
        'Total Return (%)': total_return,
        'Annualized Return (%)': annualized_return,
        'Win Rate (%)': win_rate,
        'Profit Factor': profit_factor,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown (%)': max_drawdown,
        'Average Win (%)': avg_win,
        'Average Loss (%)': avg_loss,
        'Average Trade (%)': avg_trade,
        'Expectancy (%)': expectancy,
        'Number of Trades': metrics_dict['num_trades']
    }

# Calculate metrics for all strategies
print("=" * 80)
print("COMPREHENSIVE PERFORMANCE METRICS COMPARISON")
print("=" * 80)

# SMA Strategy metrics
sma_perf = calculate_performance_metrics(sma_df, sma_metrics)
print("\nSMA 10 + 2% Filter Strategy:")
print("-" * 80)
for metric, value in sma_perf.items():
    if isinstance(value, float):
        print(f"{metric:.<40} {value:>10.2f}")
    else:
        print(f"{metric:.<40} {value:>10}")

# RSI + MACD Strategy metrics
rsi_macd_perf = calculate_performance_metrics(rsi_macd_df, rsi_macd_metrics)
print("\nRSI + MACD Combined Strategy:")
print("-" * 80)
for metric, value in rsi_macd_perf.items():
    if isinstance(value, float):
        print(f"{metric:.<40} {value:>10.2f}")
    else:
        print(f"{metric:.<40} {value:>10}")

print("\n" + "=" * 80)

In [ ]:
# Create performance comparison visualization

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

strategies = ['SMA Filter', 'RSI+MACD']
metrics_data = [sma_perf, rsi_macd_perf]

# Plot 1: Total Return Comparison
returns = [m['Total Return (%)'] for m in metrics_data]
colors = ['green' if r > 0 else 'red' for r in returns]
axes[0, 0].bar(strategies, returns, color=colors, alpha=0.7)
axes[0, 0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0, 0].set_title('Total Return Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Return (%)', fontsize=12)
axes[0, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(returns):
    axes[0, 0].text(i, v, f'{v:.1f}%', ha='center', va='bottom' if v > 0 else 'top')

# Plot 2: Win Rate Comparison
win_rates = [m['Win Rate (%)'] for m in metrics_data]
axes[0, 1].bar(strategies, win_rates, color='blue', alpha=0.7)
axes[0, 1].axhline(y=50, color='red', linestyle='--', label='50% (Random)', alpha=0.7)
axes[0, 1].set_title('Win Rate Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Win Rate (%)', fontsize=12)
axes[0, 1].set_ylim(0, 100)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(win_rates):
    axes[0, 1].text(i, v, f'{v:.1f}%', ha='center', va='bottom')

# Plot 3: Sharpe Ratio Comparison
sharpe_ratios = [m['Sharpe Ratio'] for m in metrics_data]
colors = ['green' if s > 1 else 'orange' if s > 0 else 'red' for s in sharpe_ratios]
axes[1, 0].bar(strategies, sharpe_ratios, color=colors, alpha=0.7)
axes[1, 0].axhline(y=1, color='green', linestyle='--', label='Good (>1.0)', alpha=0.7)
axes[1, 0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1, 0].set_title('Sharpe Ratio Comparison', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Sharpe Ratio', fontsize=12)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(sharpe_ratios):
    axes[1, 0].text(i, v, f'{v:.2f}', ha='center', va='bottom' if v > 0 else 'top')

# Plot 4: Maximum Drawdown Comparison
max_drawdowns = [abs(m['Max Drawdown (%)']) for m in metrics_data]  # Take absolute value
axes[1, 1].bar(strategies, max_drawdowns, color='red', alpha=0.7)
axes[1, 1].set_title('Maximum Drawdown Comparison (Lower is Better)', 
                     fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Max Drawdown (%)', fontsize=12)
axes[1, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(max_drawdowns):
    axes[1, 1].text(i, v, f'{v:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 6. Malaysian Transaction Costs

One of the most critical aspects of realistic backtesting is **including transaction costs**. Many strategies that look profitable on paper fail in live trading because they don't account for:

### Malaysian Stock Market Fees (Bursa Malaysia):

| Fee Type | Rate | Notes |
|----------|------|-------|
| **Brokerage Commission** | 0.42% | Typical online broker rate |
| **Clearing Fee** | 0.03% | Charged by Bursa |
| **Stamp Duty** | 0.10% | Government tax (max RM200 per contract) |
| **Minimum Commission** | RM2.88 | For trades < RM685.71 |

**Total Transaction Cost**: Approximately **0.55% per trade** (buy + sell = 1.1% round trip)

### Impact Example:
- Trade value: RM10,000
- Buy commission: RM42.00
- Sell commission: RM42.00
- Clearing fees: RM6.00
- Stamp duty: RM20.00
- **Total cost**: RM110.00 (1.1% of trade value)

This means you need at least **1.1% profit just to break even**!

In [ ]:
def calculate_transaction_costs(trade_value):
    """
    Calculate Malaysian stock market transaction costs.
    
    Parameters:
    - trade_value: Value of the trade in RM
    
    Returns:
    - Total transaction cost in RM
    """
    # Brokerage commission (0.42%)
    commission = trade_value * 0.0042
    commission = max(commission, 2.88)  # Minimum RM2.88
    
    # Clearing fee (0.03%)
    clearing_fee = trade_value * 0.0003
    
    # Stamp duty (0.10%, max RM200)
    stamp_duty = min(trade_value * 0.001, 200)
    
    total_cost = commission + clearing_fee + stamp_duty
    return total_cost

def backtest_with_transaction_costs(data, strategy_func, initial_capital=10000, **kwargs):
    """
    Backtest strategy with realistic Malaysian transaction costs.
    
    Parameters:
    - data: DataFrame with OHLCV data
    - strategy_func: Function that returns DataFrame with 'Position' column
    - initial_capital: Starting capital in RM
    - **kwargs: Additional arguments for strategy function
    
    Returns:
    - DataFrame with portfolio value including costs
    - Dictionary with performance metrics
    """
    # Run strategy to get signals
    df, base_metrics = strategy_func(data, initial_capital=initial_capital, **kwargs)
    
    # Recalculate portfolio value with transaction costs
    cash = initial_capital
    shares = 0
    portfolio_values = []
    total_costs = 0
    
    for i in range(len(df)):
        position = df['Position'].iloc[i]
        prev_position = df['Position'].iloc[i-1] if i > 0 else 0
        price = df['Close'].iloc[i]
        
        # Check for position change (trade execution)
        if position != prev_position:
            if position == 1 and prev_position == 0:
                # BUY: Use all cash to buy shares
                trade_value = cash
                costs = calculate_transaction_costs(trade_value)
                shares = (cash - costs) / price
                cash = 0
                total_costs += costs
            elif position == 0 and prev_position == 1:
                # SELL: Sell all shares
                trade_value = shares * price
                costs = calculate_transaction_costs(trade_value)
                cash = trade_value - costs
                shares = 0
                total_costs += costs
        
        # Calculate current portfolio value
        portfolio_value = cash + (shares * price)
        portfolio_values.append(portfolio_value)
    
    df['Portfolio_Value_With_Costs'] = portfolio_values
    
    # Calculate metrics with costs
    final_value = df['Portfolio_Value_With_Costs'].iloc[-1]
    total_return = (final_value - initial_capital) / initial_capital * 100
    
    # Calculate impact of costs
    return_without_costs = base_metrics['total_return']
    cost_impact = return_without_costs - total_return
    
    metrics_with_costs = {
        'strategy': base_metrics['strategy'] + ' (with costs)',
        'initial_capital': initial_capital,
        'final_value': final_value,
        'total_return': total_return,
        'total_costs': total_costs,
        'cost_impact': cost_impact,
        'return_without_costs': return_without_costs,
        'num_trades': base_metrics['num_trades']
    }
    
    return df, metrics_with_costs

# Backtest SMA strategy with transaction costs
sma_with_costs_df, sma_with_costs = backtest_with_transaction_costs(
    maybank_data, 
    backtest_sma_filter_strategy,
    initial_capital=10000,
    sma_period=10,
    filter_pct=2.0
)

print("=" * 80)
print("IMPACT OF TRANSACTION COSTS ON SMA STRATEGY")
print("=" * 80)
print(f"Return WITHOUT costs: {sma_with_costs['return_without_costs']:.2f}%")
print(f"Return WITH costs: {sma_with_costs['total_return']:.2f}%")
print(f"Cost impact: -{sma_with_costs['cost_impact']:.2f}%")
print(f"Total costs paid: RM{sma_with_costs['total_costs']:.2f}")
print(f"Number of trades: {sma_with_costs['num_trades']}")
print(f"Average cost per trade: RM{sma_with_costs['total_costs']/sma_with_costs['num_trades']:.2f}")
print("=" * 80)

In [ ]:
# Visualize impact of transaction costs

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Portfolio value with and without costs
ax1.plot(sma_with_costs_df.index, sma_with_costs_df['Portfolio_Value'], 
         label='Without Transaction Costs', linewidth=2, color='blue', alpha=0.7)
ax1.plot(sma_with_costs_df.index, sma_with_costs_df['Portfolio_Value_With_Costs'], 
         label='With Transaction Costs', linewidth=2, color='red')
ax1.axhline(y=10000, color='black', linestyle='--', 
            label='Initial Capital', alpha=0.5)
ax1.fill_between(sma_with_costs_df.index, 
                  sma_with_costs_df['Portfolio_Value_With_Costs'],
                  sma_with_costs_df['Portfolio_Value'],
                  alpha=0.3, color='red', label='Cost Impact')
ax1.set_title('Impact of Transaction Costs on Portfolio Value', 
              fontsize=16, fontweight='bold')
ax1.set_ylabel('Portfolio Value (RM)', fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Cost breakdown
trade_values = [5000, 10000, 20000, 50000, 100000]
costs = [calculate_transaction_costs(tv) for tv in trade_values]
cost_percentages = [c/tv*100 for c, tv in zip(costs, trade_values)]

ax2.plot(trade_values, cost_percentages, marker='o', linewidth=2, markersize=8)
ax2.axhline(y=1.1, color='red', linestyle='--', 
            label='Round-trip cost (~1.1%)', alpha=0.7)
ax2.set_title('Transaction Cost as Percentage of Trade Value', 
              fontsize=16, fontweight='bold')
ax2.set_xlabel('Trade Value (RM)', fontsize=12)
ax2.set_ylabel('Cost (%)', fontsize=12)
ax2.set_xscale('log')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("Transaction costs have a SIGNIFICANT impact on strategy performance!")
print("Always include costs in backtests to get realistic results.")

## 7. Realistic Testing: Slippage and Walk-Forward Analysis

Beyond transaction costs, we need to account for:

### Slippage
**Slippage** is the difference between expected trade price and actual execution price. Causes:
- Market orders execute at current market price (not yesterday's close)
- Bid-ask spread
- Low liquidity
- Fast-moving markets

**Typical slippage**: 0.1-0.5% for liquid Malaysian stocks

### Walk-Forward Testing
**Walk-forward testing** helps avoid overfitting by:
1. Dividing data into multiple periods
2. Training (optimizing) on one period
3. Testing on the next period
4. Rolling forward and repeating

This simulates how strategies perform on **unseen future data**.

In [ ]:
def walk_forward_test(data, strategy_func, train_months=12, test_months=3, **kwargs):
    """
    Perform walk-forward testing on a strategy.
    
    Parameters:
    - data: DataFrame with OHLCV data
    - strategy_func: Strategy function to test
    - train_months: Months of data for training/optimization
    - test_months: Months of data for testing
    - **kwargs: Additional strategy parameters
    
    Returns:
    - List of test period results
    - Combined DataFrame with all test periods
    """
    results = []
    all_test_data = []
    
    # Calculate window sizes in days (approximate)
    train_days = train_months * 21  # ~21 trading days per month
    test_days = test_months * 21
    
    start_idx = 0
    test_num = 1
    
    while start_idx + train_days + test_days < len(data):
        # Split into train and test sets
        train_end_idx = start_idx + train_days
        test_end_idx = train_end_idx + test_days
        
        train_data = data.iloc[start_idx:train_end_idx]
        test_data = data.iloc[train_end_idx:test_end_idx]
        
        # Run strategy on test period
        # (In real walk-forward, we'd optimize on train_data and test on test_data)
        test_df, test_metrics = strategy_func(test_data, **kwargs)
        
        results.append({
            'test_num': test_num,
            'train_start': train_data.index[0],
            'train_end': train_data.index[-1],
            'test_start': test_data.index[0],
            'test_end': test_data.index[-1],
            'return': test_metrics['total_return'],
            'num_trades': test_metrics['num_trades']
        })
        
        all_test_data.append(test_df)
        
        # Move window forward by test period
        start_idx = train_end_idx
        test_num += 1
    
    results_df = pd.DataFrame(results)
    return results_df, all_test_data

# Perform walk-forward test on SMA strategy
wf_results, wf_data = walk_forward_test(
    maybank_data,
    backtest_sma_filter_strategy,
    train_months=12,
    test_months=3,
    initial_capital=10000,
    sma_period=10,
    filter_pct=2.0
)

print("=" * 80)
print("WALK-FORWARD TEST RESULTS (SMA Strategy)")
print("=" * 80)
print(f"Number of test periods: {len(wf_results)}")
print(f"\nTest Period Results:")
print(wf_results.to_string(index=False))
print("\n" + "=" * 80)
print(f"Average return across periods: {wf_results['return'].mean():.2f}%")
print(f"Standard deviation: {wf_results['return'].std():.2f}%")
print(f"Best period: {wf_results['return'].max():.2f}%")
print(f"Worst period: {wf_results['return'].min():.2f}%")
print(f"Profitable periods: {len(wf_results[wf_results['return'] > 0])}/{len(wf_results)}")
print("=" * 80)

In [ ]:
# Visualize walk-forward test results

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Returns by test period
colors = ['green' if r > 0 else 'red' for r in wf_results['return']]
ax1.bar(wf_results['test_num'], wf_results['return'], color=colors, alpha=0.7)
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax1.axhline(y=wf_results['return'].mean(), color='blue', linestyle='--', 
            label=f'Average: {wf_results["return"].mean():.2f}%', linewidth=2)
ax1.set_title('Walk-Forward Test: Returns by Period', fontsize=16, fontweight='bold')
ax1.set_xlabel('Test Period', fontsize=12)
ax1.set_ylabel('Return (%)', fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Return distribution
ax2.hist(wf_results['return'], bins=10, edgecolor='black', alpha=0.7)
ax2.axvline(x=wf_results['return'].mean(), color='blue', linestyle='--', 
            label=f'Mean: {wf_results["return"].mean():.2f}%', linewidth=2)
ax2.axvline(x=0, color='red', linestyle='-', linewidth=1, alpha=0.5)
ax2.set_title('Distribution of Returns Across Test Periods', fontsize=16, fontweight='bold')
ax2.set_xlabel('Return (%)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nWalk-Forward Testing Insight:")
print("This shows how the strategy performs on UNSEEN data periods.")
print("Consistent performance across periods = robust strategy.")
print("High variance = strategy may be overfit to specific market conditions.")

## 8. Strategy Parameter Optimization

**Parameter optimization** finds the best settings for a strategy. However, it comes with serious risks:

### The Danger of Overfitting:

```
Good Optimization          vs          Overfitting (Curve-Fitting)
├─ Few parameters                     ├─ Many parameters
├─ Wide optimal ranges                ├─ Narrow optimal ranges
├─ Robust across periods              ├─ Fails on new data
├─ Based on logic                     ├─ Based on historical accidents
└─ Out-of-sample validation           └─ Only in-sample testing
```

### Best Practices:
1. **Optimize on training data, validate on test data**
2. **Use walk-forward testing**
3. **Limit number of parameters** (2-3 maximum)
4. **Look for robust regions**, not single optimal values
5. **Require economic/technical rationale** for parameter choices

In [ ]:
def optimize_sma_strategy(data, sma_range=range(5, 31, 5), 
                         filter_range=np.arange(0, 5.1, 1.0)):
    """
    Optimize SMA strategy parameters using grid search.
    
    WARNING: This is for educational purposes. In practice, you must:
    1. Split data into train/test sets
    2. Optimize on train set only
    3. Validate on test set
    4. Use walk-forward testing
    
    Parameters:
    - data: DataFrame with OHLCV data
    - sma_range: Range of SMA periods to test
    - filter_range: Range of filter percentages to test
    
    Returns:
    - DataFrame with optimization results
    """
    results = []
    
    for sma_period in sma_range:
        for filter_pct in filter_range:
            try:
                df, metrics = backtest_sma_filter_strategy(
                    data,
                    sma_period=sma_period,
                    filter_pct=filter_pct,
                    initial_capital=10000
                )
                
                results.append({
                    'SMA_Period': sma_period,
                    'Filter_Pct': filter_pct,
                    'Total_Return': metrics['total_return'],
                    'Num_Trades': metrics['num_trades']
                })
            except:
                # Skip parameter combinations that cause errors
                continue
    
    results_df = pd.DataFrame(results)
    return results_df.sort_values('Total_Return', ascending=False)

# Optimize SMA strategy (on full dataset - NOT RECOMMENDED in practice!)
print("⚠️  WARNING: Optimizing on full dataset for demonstration only!")
print("In practice, ALWAYS split into train/test and use walk-forward testing.\n")

optimization_results = optimize_sma_strategy(maybank_data)

print("=" * 80)
print("SMA STRATEGY OPTIMIZATION RESULTS (Top 10)")
print("=" * 80)
print(optimization_results.head(10).to_string(index=False))
print("\n" + "=" * 80)

# Get best parameters
best = optimization_results.iloc[0]
print(f"\nBest Parameters (on this dataset):")
print(f"SMA Period: {int(best['SMA_Period'])}")
print(f"Filter %: {best['Filter_Pct']:.1f}%")
print(f"Total Return: {best['Total_Return']:.2f}%")
print(f"Number of Trades: {int(best['Num_Trades'])}")
print("\n⚠️  Remember: These are OVERFIT to this specific dataset!")
print("They will likely perform worse on future data.")

In [ ]:
# Visualize optimization results as heatmap

# Create pivot table for heatmap
heatmap_data = optimization_results.pivot_table(
    values='Total_Return',
    index='Filter_Pct',
    columns='SMA_Period',
    aggfunc='mean'
)

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn', 
            center=0, linewidths=0.5, cbar_kws={'label': 'Total Return (%)'})
plt.title('SMA Strategy Optimization: Total Return by Parameters', 
          fontsize=16, fontweight='bold')
plt.xlabel('SMA Period (days)', fontsize=12)
plt.ylabel('Filter Percentage (%)', fontsize=12)
plt.tight_layout()
plt.show()

print("\nHeatmap Interpretation:")
print("- Green areas: Profitable parameter combinations")
print("- Red areas: Unprofitable parameter combinations")
print("- Look for GREEN REGIONS (robust), not single best cells")
print("- Single optimal cell = likely overfit")
print("- Broad optimal region = potentially robust strategy")

## Practice Exercises

Test your understanding with these backtesting challenges:

### Exercise 1: Multi-Stock Backtest

Backtest the SMA 10 + 2% filter strategy on THREE Malaysian stocks:
- Maybank (1155.KL)
- Public Bank (1295.KL)
- Gamuda (5398.KL)

Compare their performance. Which stock performed best? Why do you think that is?

**Your Code Here:**

In [ ]:
# Exercise 1: Your solution

# Step 1: Fetch data for all three stocks


# Step 2: Backtest each stock


# Step 3: Compare results and visualize


### Exercise 2: Transaction Cost Sensitivity

Create a function that shows how different commission rates affect strategy profitability.

Test commission rates from 0% to 1% in 0.1% increments. Plot the results.

**Question**: At what commission rate does the SMA strategy become unprofitable?

**Your Code Here:**

In [ ]:
# Exercise 2: Your solution

# Create function to test different commission rates


# Plot results


### Exercise 3: Create Your Own Combined Strategy

Design and backtest your own strategy combining ANY indicators from previous modules:
- Moving Averages (SMA, EMA)
- RSI
- MACD
- Bollinger Bands

**Requirements**:
1. Clear entry and exit rules
2. Include transaction costs
3. Calculate all performance metrics
4. Compare with buy-and-hold

**Your Code Here:**

In [ ]:
# Exercise 3: Your solution

# Define your strategy rules as comments first


# Implement the strategy


# Calculate metrics and compare with buy-and-hold


### Exercise 4: Robust Parameter Selection

Using the optimization results from Section 8, identify a "robust region" of parameters rather than a single optimal value.

**Tasks**:
1. Find all parameter combinations with returns > 15%
2. What is the most common SMA period in these combinations?
3. What is the most common filter percentage?
4. Test this "robust" combination on a different time period

**Your Code Here:**

In [ ]:
# Exercise 4: Your solution

# Analyze optimization results


# Test robust parameters on different period


## Summary

Congratulations! You've learned how to build and evaluate trading strategies using proper backtesting frameworks.

### Key Concepts Covered:

1. ✅ **Backtesting Importance**: Test before you invest real money
2. ✅ **Simple Framework**: Buy-and-hold baseline for comparison
3. ✅ **SMA + Filter Strategy**: Malaysian research showing 24-35% returns
4. ✅ **RSI + MACD Strategy**: Combined indicators for 73% win rate
5. ✅ **Performance Metrics**: Win rate, profit factor, Sharpe ratio, max drawdown
6. ✅ **Transaction Costs**: Malaysian fees (~1.1% round trip) significantly impact results
7. ✅ **Walk-Forward Testing**: Avoid overfitting with proper validation
8. ✅ **Parameter Optimization**: Find robust regions, not single optimal values

### Backtesting Best Practices:

| ✅ DO | ❌ DON'T |
|-------|----------|
| Include transaction costs | Ignore fees and slippage |
| Use walk-forward testing | Optimize on entire dataset |
| Test on multiple stocks | Cherry-pick best results |
| Calculate multiple metrics | Rely only on total return |
| Look for robust regions | Trust single optimal parameters |
| Set realistic expectations | Expect past results to repeat |
| Consider market conditions | Assume markets never change |
| Document your assumptions | Hide limitations |

### Critical Warnings:

⚠️ **Past performance does not guarantee future results**

⚠️ **Overfitting is the #1 cause of strategy failure**

⚠️ **Always include transaction costs - they can make or break profitability**

⚠️ **Test on out-of-sample data before risking real money**

⚠️ **Markets change - strategies that worked before may not work tomorrow**

### Next Steps:

1. **Module 09**: Risk Management and Position Sizing
2. **Module 10**: Building a Complete Trading System

### Additional Resources:

- **Book**: "Advances in Financial Machine Learning" by Marcos López de Prado
- **Library**: `backtrader` - Professional Python backtesting framework
- **Library**: `zipline` - Algorithmic trading library
- **Research**: Bursa Malaysia research papers on Malaysian market characteristics

---

**Remember**: A good backtest doesn't guarantee success, but a bad backtest guarantees failure. Test thoroughly, trade carefully! 📊